In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import math as math
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from datetime import date
from datetime import datetime

In [2]:
def agg_string_to_arr(arr):
    new_arr = []
    for e in arr:
        if e not in new_arr:
            new_arr.append(e)
    return new_arr

def attendance_demog_wash_by_child(attendance, demog):
    #-explain
    
    #CSV -> DF
    attendance_df = pd.read_csv(attendance)
    demog_df = pd.read_csv(demog)
    
    #ATTENDANCE
    #----------
    
    #Extrapolate attendance specifier as boolean columns
    one_hot_atten = pd.get_dummies(attendance_df['Attendance'])
    one_hot_sesh = pd.get_dummies(attendance_df['Session'])
    attendance_clean = attendance_df.join(one_hot_atten)
    attendance_clean = attendance_clean.join(one_hot_sesh)
    
    #Combine rows by ChildID and 
    attendance_clean = attendance_clean.groupby(['ChildID'],as_index=False).agg(
             {'Best Interest': 'sum',
              'In Attendance': 'sum',
              'Other': 'sum',
              'Sick':'sum',
              'Unexcused Absence':'sum',
              'FD':'sum',
              'PD':'sum',
              'Center':agg_string_to_arr})
    
    #Add aggregate columns: Total Attendance,Absence, and Days AND Attendance Rate
    attendance_clean['Total Attendance'] = attendance_clean['In Attendance']
    attendance_clean['Total Absence'] = attendance_clean['Other'] + attendance_clean['Sick'] + attendance_clean['Unexcused Absence'] + attendance_clean['Best Interest']
    attendance_clean['Total Days'] = attendance_clean['Total Attendance'] + attendance_clean['Total Absence']
    attendance_clean['Attendance Rate'] = attendance_clean['Total Attendance'] / attendance_clean['Total Days']
    
    attendance_clean_final = attendance_clean
    
    #DEMOGRAPHICS
    #------------
    
    #Reorder Columns
    demog_clean = demog_df[['ChildID','ChildEthnicity','ChildRace','ChildLanguage','ChildFlags','ChildDateofBirth']]
    
    #Drop NAN 
    demog_clean = demog_clean[demog_clean['ChildDateofBirth'].notna()]

    #Add current age column
    today = datetime.today() 

    def get_age(bday_str):
        datetime_bday = datetime.strptime(str(bday_str),"%m/%d/%Y")
        age = (today-datetime_bday).days // 365
        return age

    demog_clean['Age (as of current day)'] = demog_clean['ChildDateofBirth'].apply(get_age)
    
    #Make ID# an int
    demog_clean['ChildID'] = demog_clean.ChildID.astype(int)
    
    #Aggregate rows to combine duplicate ChildIDs
    
    #Df with only aggregated string columns
    combined_str = demog_clean.groupby('ChildID').agg({'ChildRace':agg_string_to_arr, 'ChildFlags':agg_string_to_arr})
    
    #Combine agg str with demog clean
    merged = combined_str.merge(demog_clean, on = 'ChildID',how = 'left')
    
    #Drop extraneous columns 
    cleaned = merged[['ChildID','ChildEthnicity','ChildLanguage','ChildRace_x','ChildFlags_x','ChildDateofBirth','Age (as of current day)']]
    
    #Drop duplicates rows
    demog_clean_final = cleaned.drop_duplicates(subset= 'ChildID')
    
    #Rename columns
    demog_clean_final = demog_clean_final.rename(columns = {"ChildRace_x":'ChildRace',"ChildFlags_x":'ChildFlags'})
    
    
    #JOIN DFs for MASTER
    master = pd.merge(demog_clean_final,attendance_clean_final, how='inner', on= 'ChildID')
    
    #EXPORT
    #------
    from pathlib import Path 

    filepath = Path(attendance[:-4] + 'CHILDID_out.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    master.to_csv(filepath)
    
    return master

df = attendance_demog_wash_by_child('18-19 attendance master.csv','PvH-demographicsdash_childdata.csv')
df


,ChildID,ChildEthnicity,ChildLanguage,ChildRace,ChildFlags,ChildDateofBirth,Age (as of current day),Best Interest,In Attendance,Other,Sick,Unexcused Absence,FD,PD,Center,Total Attendance,Total Absence,Total Days,Attendance Rate
0,18578,Hispanic or Latino,01 -- Spanish,[Caucasian],[Preschool],12/28/2004,17,0,102.0,0,0,0,102.0,0,[FH],102.0,0,102.0,1.000000
1,22717,Hispanic or Latino,01 -- Spanish,[Caucasian],[Infant/Toddler],01/13/2008,14,0,245.0,2,1,0,248.0,0,[FH],245.0,3,248.0,0.987903
2,25173,Hispanic or Latino,00 -- English,[Caucasian],[Preschool],01/06/2007,15,0,242.0,2,4,0,248.0,0,[FH],242.0,6,248.0,0.975806
3,30829,Hispanic or Latino,01 -- Spanish,[Caucasian],[Infant/Toddler],11/19/2010,11,0,246.0,2,0,0,248.0,0,[FH],246.0,2,248.0,0.991935
4,31583,Hispanic or Latino,00 -- English,[Caucasian],[Emergency Care],05/22/2010,12,3,232.0,1,10,0,246.0,0,[Linda Vista],232.0,14,246.0,0.943089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4781,65271,Not Hispanic or Latino,00 -- English,[Caucasian],"[Admissions Agreement #15 (check box if Yes), ...",03/06/2015,7,0,2.0,0,0,0,0.0,2,[Emerald Vista],2.0,0,2.0,1.000000
4782,65289,Not Hispanic or Latino,00 -- English,[Caucasian],[nan],08/15/2016,6,0,1.0,0,0,0,1.0,0,[Esther Hobbs],1.0,0,1.0,1.000000
4783,65290,Hispanic or Latino,00 -- English,[nan],"[Shelter-In-Place, IFSP]",11/14/2017,4,0,3.0,0,0,0,3.0,0,[Esther Hobbs],3.0,0,3.0,1.000000
4784,65301,Hispanic or Latino,01 -- Spanish,[Caucasian],"[IEP, Admissions Agreement #16 (check box if Y...",05/12/2016,6,0,1.0,0,0,0,1.0,0,[Castlemont],1.0,0,1.0,1.000000


In [3]:
def attendance_demog_wash_by_center(attendance, demog):
    #-explain
    
    #CSV -> DF
    attendance_df = pd.read_csv(attendance)
    demog_df = pd.read_csv(demog)
    
    #ATTENDANCE
    #----------
    
    #Extrapolate attendance specifier as boolean columns
    one_hot_atten = pd.get_dummies(attendance_df['Attendance'])
    one_hot_sesh = pd.get_dummies(attendance_df['Session'])
    attendance_clean = attendance_df.join(one_hot_atten)
    attendance_clean = attendance_clean.join(one_hot_sesh)
    
    #Combine rows by ChildID and 
    attendance_clean = attendance_clean.groupby(['ChildID','Center'],as_index=False).agg(
             {'Best Interest': 'sum',
              'In Attendance': 'sum',
              'Other': 'sum',
              'Sick':'sum',
              'Unexcused Absence':'sum',
              'FD':'sum',
              'PD':'sum'})
    
    #Add aggregate columns: Total Attendance,Absence, and Days AND Attendance Rate
    attendance_clean['Total Attendance'] = attendance_clean['In Attendance']
    attendance_clean['Total Absence'] = attendance_clean['Other'] + attendance_clean['Sick'] + attendance_clean['Unexcused Absence'] + attendance_clean['Best Interest']
    attendance_clean['Total Days'] = attendance_clean['Total Attendance'] + attendance_clean['Total Absence']
    attendance_clean['Attendance Rate'] = attendance_clean['Total Attendance'] / attendance_clean['Total Days']
    
    attendance_clean_final = attendance_clean
    
    #DEMOGRAPHICS
    #------------
    
    #Reorder Columns
    demog_clean = demog_df[['ChildID','ChildEthnicity','ChildRace','ChildLanguage','ChildFlags','ChildDateofBirth']]
    
    #Drop NAN 
    demog_clean = demog_clean[demog_clean['ChildDateofBirth'].notna()]

    #Add current age column
    today = datetime.today() 

    def get_age(bday_str):
        datetime_bday = datetime.strptime(str(bday_str),"%m/%d/%Y")
        age = (today-datetime_bday).days // 365
        return age

    demog_clean['Age (as of current day)'] = demog_clean['ChildDateofBirth'].apply(get_age)
    
    #Make ID# an int
    demog_clean['ChildID'] = demog_clean.ChildID.astype(int)
    
    #Aggregate rows to combine duplicate ChildIDs
    
    #Df with only aggregated string columns
    combined_str = demog_clean.groupby('ChildID').agg({'ChildRace':agg_string_to_arr, 'ChildFlags':agg_string_to_arr})
    
    #Combine agg str with demog clean
    merged = combined_str.merge(demog_clean, on = 'ChildID',how = 'left')
    
    #Drop extraneous columns 
    cleaned = merged[['ChildID','ChildEthnicity','ChildLanguage','ChildRace_x','ChildFlags_x','ChildDateofBirth','Age (as of current day)']]
    
    #Drop duplicates rows
    demog_clean_final = cleaned.drop_duplicates(subset= 'ChildID')
    
    #Rename columns
    demog_clean_final = demog_clean_final.rename(columns = {"ChildRace_x":'ChildRace',"ChildFlags_x":'ChildFlags'})
    
    
    #JOIN DFs for MASTER
    master = pd.merge(demog_clean_final,attendance_clean_final, how='inner', on= 'ChildID')
    
    #EXPORT
    #------
    from pathlib import Path 

    filepath = Path(attendance[:-4] + 'CENTER_out.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    master.to_csv(filepath)
    
    return master


In [ ]:
center_df =attendance_demog_wash_by_center('18-19 attendance master.csv','PvH-demographicsdash_childdata.csv')
center_df['Center'].value_counts().head()

In [9]:
attendance_demog_wash_by_center('21-22 attendance master.csv', 'PvH-demographicsdash_childdata.csv')

,ChildID,ChildEthnicity,ChildLanguage,ChildRace,ChildFlags,ChildDateofBirth,Age (as of current day),Center,Best Interest,In Attendance,Other,Sick,Unexcused Absence,FD,PD,Total Attendance,Total Absence,Total Days,Attendance Rate
0,31583,Hispanic or Latino,00 -- English,[Caucasian],[Emergency Care],05/22/2010,12,Linda Vista,0,3.0,0,0,3,104.0,0.0,3.0,3,6.0,0.500000
1,37814,Hispanic or Latino,00 -- English,[Caucasian],"[Admissions Agreement #14 (check box if Yes), ...",09/22/2009,13,Linda Vista,1,94.0,27,2,0,142.0,0.0,94.0,30,124.0,0.758065
2,37815,Hispanic or Latino,00 -- English,[Caucasian],"[Admissions Agreement #15 (check box if Yes), ...",01/01/2012,10,Linda Vista,9,254.0,57,17,0,355.0,0.0,254.0,83,337.0,0.753709
3,40481,Hispanic or Latino,00 -- English,[Caucasian],[nan],03/08/2013,9,Linda Vista,5,237.0,6,17,0,347.0,0.0,237.0,28,265.0,0.894340
4,40605,Hispanic or Latino,00 -- English,[Caucasian],[Emergency Care],10/12/2010,12,Linda Vista,18,225.0,34,64,0,341.0,0.0,225.0,116,341.0,0.659824
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5398,75331,Hispanic or Latino,01 -- Spanish,[Caucasian],[Limited English],06/12/2019,3,Decoto,0,0.0,0,0,10,0.0,10.0,0.0,10,10.0,0.000000
5399,75332,Hispanic or Latino,00 -- English,[Caucasian],"[Admissions Agreement #15 (check box if Yes), ...",04/30/2019,3,Goss,1,2.0,5,0,0,0.0,8.0,2.0,6,8.0,0.250000
5400,75334,Hispanic or Latino,00 -- English,[Caucasian],"[Infant/Toddler, Admissions Agreement #16 (che...",01/11/2022,0,FH,4,0.0,0,0,0,4.0,0.0,0.0,4,4.0,0.000000
5401,75365,Hispanic or Latino,01 -- Spanish,[Caucasian],"[Infant/Toddler, Admissions Agreement #14 (che...",03/27/2020,2,Arbuckle,0,4.0,0,0,0,4.0,0.0,4.0,0,4.0,1.000000


In [11]:
def combine_years(year, another_year,name):
    year_df = pd.read_csv(year)
    a_year_df = pd.read_csv(another_year)
    
    combined = pd.concat([year_df, a_year_df])
    
    from pathlib import Path 

    filepath = Path(name + '.csv')  
    filepath.parent.mkdir(parents=True, exist_ok=True)  
    combined.to_csv(filepath)
    
    return combined

In [12]:
combine_years('18-19 attendance masterCENTER_out.csv','19-20 attendance masterCENTER_out.csv','18-20_by_center')

,Unnamed: 0,ChildID,ChildEthnicity,ChildLanguage,ChildRace,ChildFlags,ChildDateofBirth,Age (as of current day),Center,Best Interest,In Attendance,Other,Sick,Unexcused Absence,FD,PD,Total Attendance,Total Absence,Total Days,Attendance Rate
0,0,18578,Hispanic or Latino,01 -- Spanish,['Caucasian'],['Preschool'],12/28/2004,17,FH,0,102.0,0,0,0,102.0,0,102.0,0,102.0,1.000000
1,1,22717,Hispanic or Latino,01 -- Spanish,['Caucasian'],['Infant/Toddler'],01/13/2008,14,FH,0,245.0,2,1,0,248.0,0,245.0,3,248.0,0.987903
2,2,25173,Hispanic or Latino,00 -- English,['Caucasian'],['Preschool'],01/06/2007,15,FH,0,242.0,2,4,0,248.0,0,242.0,6,248.0,0.975806
3,3,30829,Hispanic or Latino,01 -- Spanish,['Caucasian'],['Infant/Toddler'],11/19/2010,11,FH,0,246.0,2,0,0,248.0,0,246.0,2,248.0,0.991935
4,4,31583,Hispanic or Latino,00 -- English,['Caucasian'],['Emergency Care'],05/22/2010,12,Linda Vista,3,232.0,1,10,0,246.0,0,232.0,14,246.0,0.943089
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5110,5110,68331,Hispanic or Latino,00 -- English,['Caucasian'],[nan],10/11/2014,8,FH,0,22.0,0,0,0,22.0,0,22.0,0,22.0,1.000000
5111,5111,68332,Hispanic or Latino,00 -- English,['Caucasian'],['Admissions Agreement #16 (check box if Yes)'...,06/11/2016,6,FH,0,22.0,0,0,0,22.0,0,22.0,0,22.0,1.000000
5112,5112,68333,Hispanic or Latino,00 -- English,['Caucasian'],['Admissions Agreement #16 (check box if Yes)'...,07/07/2016,6,FH,0,22.0,0,0,0,22.0,0,22.0,0,22.0,1.000000
5113,5113,68334,Hispanic or Latino,00 -- English,['Caucasian'],[nan],05/09/2016,6,FH,0,22.0,0,0,0,22.0,0,22.0,0,22.0,1.000000


In [31]:
combined_18to22 = combine_years('18-20_by_center.csv','21-22 attendance masterCENTER_out.csv','18-22_by_center').iloc[:,2:]

In [32]:
from pathlib import Path 

filepath = Path('18-22_by_center' + '.csv')  
filepath.parent.mkdir(parents=True, exist_ok=True)  
combined_18to22.to_csv(filepath)

